In [ ]:
from google.colab import drive

drive.mount('drive', force_remount=True)

# введите имя папки на гугл диске, в которую вы сохранили папку vision,
# содержащую .py файлы, а также папки datatasets и classifiers например,
# cv/assignments/assignment1/vision/
FOLDERNAME = None

assert FOLDERNAME is not None, "[!] Enter the foldername."

%cd drive/My Drive
%cp -r $FOLDERNAME ../../
%cd ../../
%cd vision/datasets/
!bash get_datasets.sh
%cd ../../

# Реализация нейронной сети
В этой лабораторной вам нужно будет:
* разработать нейронную сеть с полносвязными слоями, которая будет выполнять слассификацию изображений
* протестировать сеть на изображениях из выборки CIFAR-10

In [ ]:
# Этот код нужно запустить для предватирельных настроек

import numpy as np
import matplotlib.pyplot as plt

from vision.classifiers.neural_net import TwoLayerNet

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # размер изображений
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

Мы будем использовать класс `TwoLayerNet` из файла `vision/classifiers/neural_net.py` для представления нашей нейронной сети. Параметры сети хранятся в переменной класса `self.params`, где ключ - это строка с именем параметра, а значение - массивы (numpy arrays). Ниже мы создаем "грушечную" модель и "игрушенчные" данные, которые мы будем использовать при разработке нашей модели.

In [ ]:
# Создание игрушесной модели и игрушечных данных для разработки
# параметер random.seed устанавливается для того, чтобы результаты эеспериментов были повторяемыми

input_size = 4
hidden_size = 10
num_classes = 3
num_inputs = 5

def init_toy_model():
    np.random.seed(0)
    return TwoLayerNet(input_size, hidden_size, num_classes, std=1e-1)

def init_toy_data():
    np.random.seed(1)
    X = 10 * np.random.randn(num_inputs, input_size)
    y = np.array([0, 1, 2, 2, 1])
    return X, y

net = init_toy_model()
X, y = init_toy_data()

# Прямой проход: вычислите оценки для классов
Реализуйте первую часть функции `TwoLayerNet.loss`, которая вычисляет оценки для входных изображений.

Эта функция находится в файле `vision/classifiers/neural_net.py`, и она очень похожа на функции потерь SVM и Softmax: функция берет данные и параметры и вычисляет оценки для классов, значение функции потерь, градиент функции.

In [ ]:
scores = net.loss(X)
print('Your scores:')
print(scores)
print()
print('correct scores:')
correct_scores = np.asarray([
  [-0.81233741, -1.27654624, -0.70335995],
  [-0.17129677, -1.18803311, -0.47310444],
  [-0.51590475, -1.01354314, -0.8504215 ],
  [-0.15419291, -0.48629638, -0.52901952],
  [-0.00618733, -0.12435261, -0.15226949]])
print(correct_scores)
print()

# Разница должны быть очень маленькой, ~ 1e-5 - 1e-7 (диапазон может отличаться)
print('Difference between your scores and correct scores:')
print(np.sum(np.abs(scores - correct_scores)))

# Прямой проход: вычисление значения функции потерь
В этой же функции, `TwoLayerNet.loss`, реализуйте вторую часть, которая вычисляет значение потери.

In [ ]:
loss, _ = net.loss(X, y, reg=0.05)
correct_loss = 1.30378789133

# Разница должны быть очень маленькой, ~ 1e-10 - 1e-12 (диапазон может отличаться)
print('Difference between your loss and correct loss:')
print(np.sum(np.abs(loss - correct_loss)))

# Обратный проход
Реализуйте функцию до конца. Вам нужно вычислить градиент функции потерь относительно переменных `W1`, `b1`, `W2`, и  `b2`. Имея правильно реализованный прямой проход вы можете отладить обратный проход, сравнивая результаты с численым градиентом:

In [ ]:
from vision.gradient_check import eval_numerical_gradient

# С помощью градиента, полученного численно, проверьте свою реализацию градиента
# Если ваша реализиция верна, разница между вашим градиентом и градиентом,
# полученным численно должна быть меньше 1e-8 для градиента по каждой из переменных
# W1, W2, b1, and b2.

loss, grads = net.loss(X, y, reg=0.05)

for param_name in grads:
    f = lambda W: net.loss(X, y, reg=0.05)[0]
    param_grad_num = eval_numerical_gradient(f, net.params[param_name], verbose=False)
    print('%s max relative error: %e' % (param_name, rel_error(param_grad_num, grads[param_name])))

# Обучение сети
Для обучения сети мы будем использовать метод стохастического градиентного спуска (stochastic gradient descent, SGD), так же как для классификаторов SVM и Softmax. Допишите пропущенные куски кода в функции `TwoLayerNet.train`. Также вам понадобится реализовать функцию `TwoLayerNet.predict`, так как в процессе обучения вы будете вызывать эту функцию для того чтобы отслеживать точность классификации.

Реализовав метод, запустите код ниже для того, чтобы обучить двухслойную сеть на игрушечных данных. Вы должны получить значение потери меньше чем 0,02.

In [ ]:
net = init_toy_model()
stats = net.train(X, y, X, y,
            learning_rate=1e-1, reg=5e-6,
            num_iters=100, verbose=False)

print('Final training loss: ', stats['loss_history'][-1])

# отобразим график изменения значения функции потерь с течением времени
plt.plot(stats['loss_history'])
plt.xlabel('iteration')
plt.ylabel('training loss')
plt.title('Training Loss history')
plt.show()

# Загрузка данных
Теперь, когда вы реализовали сеть, которая проходит все предыдущие тесты, вам необходимо загрузить данные из уже знакомой выборки CIFAR-10 и проверить работу сети.

In [ ]:
from vision.data_utils import load_CIFAR10

def get_CIFAR10_data(num_training=49000, num_validation=1000, num_test=1000):
    """
    Load the CIFAR-10 dataset from disk and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.  
    """
    # Загрузка выборки CIFAR-10 (код ниже уже знаком вам из предыдущих лабораторных)
    cifar10_dir = 'vision/datasets/cifar-10-batches-py'
    
    try:
       del X_train, y_train
       del X_test, y_test
       print('Clear previously loaded data.')
    except:
       pass

    X_train, y_train, X_test, y_test = load_CIFAR10(cifar10_dir)
        
    mask = list(range(num_training, num_training + num_validation))
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = list(range(num_training))
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = list(range(num_test))
    X_test = X_test[mask]
    y_test = y_test[mask]

    mean_image = np.mean(X_train, axis=0)
    X_train -= mean_image
    X_val -= mean_image
    X_test -= mean_image

    X_train = X_train.reshape(num_training, -1)
    X_val = X_val.reshape(num_validation, -1)
    X_test = X_test.reshape(num_test, -1)

    return X_train, y_train, X_val, y_val, X_test, y_test


X_train, y_train, X_val, y_val, X_test, y_test = get_CIFAR10_data()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

# Обучение сети
Для обучения сети мы используем метод градиентного спуска. Дополнительно мы будем снижать скорость обучения по мере увеличения количества итераций, чтобы "не проскочить" минимальное значение функции потерь. Для этого после каждой эпохи мы будем умножать скорость обучения на снижающий коэффициент.

In [ ]:
input_size = 32 * 32 * 3
hidden_size = 50
num_classes = 10
net = TwoLayerNet(input_size, hidden_size, num_classes)

# Обучение сети
stats = net.train(X_train, y_train, X_val, y_val,
            num_iters=1000, batch_size=200,
            learning_rate=1e-4, learning_rate_decay=0.95,
            reg=0.25, verbose=True)

# Тестирование на проверочной выборке
val_acc = (net.predict(X_val) == y_val).mean()
print('Validation accuracy: ', val_acc)


# Отладка обучения
С параметрами по умолчанию точность классификации на проверочной выборке должна составлять примерно 0.29, что не очень хорошо.

Один из способов понять причину того, что модель плохо обучается - это вывести графики для функции потерь и точности в зависимости от номера итерации во время обучения. Точность будем выводить и для обучающей, и для проверочной выборок.

Другой способ - визуализировать патаметры, которые мы изучили на первом слое сети. В большинстве сетей, обученных на изображениях первый слой позволяет увидеть какие-то труктурные элементы изображений из обучаюзей выборки.

In [ ]:
# График функции потерь и точность для обучеющей и проверочной выборок
plt.subplot(2, 1, 1)
plt.plot(stats['loss_history'])
plt.title('Loss history')
plt.xlabel('Iteration')
plt.ylabel('Loss')

plt.subplot(2, 1, 2)
plt.plot(stats['train_acc_history'], label='train')
plt.plot(stats['val_acc_history'], label='val')
plt.title('Classification accuracy history')
plt.xlabel('Epoch')
plt.ylabel('Classification accuracy')
plt.legend()
plt.show()

In [ ]:
from vision.vis_utils import visualize_grid

# Визуализация весов сети

def show_net_weights(net):
    W1 = net.params['W1']
    W1 = W1.reshape(32, 32, 3, -1).transpose(3, 0, 1, 2)
    plt.imshow(visualize_grid(W1, padding=3).astype('uint8'))
    plt.gca().axis('off')
    plt.show()

show_net_weights(net)

# Поиск гиперпараметров

**Причины:** Графики выше показывают, что потеря уменьшается линейно, что говорит о том, что скорость обучения может быть слишком низкой. Кроме того, нет разницы между точностью на обучающей и проверочной выборках, отсюда можно предположить, что модель имеет недостаточно параметров, и нужно увеличить их количество (можель плохо изучила признаки тренировочной выборки). С другой стороны, модель с очень большим количеством параметров склонна к переобучению, что может привести к очень большой разнице в точности между обучающей и проверочной выборками.

**Поиск:** Поиск гиперпараметров и развитие понимания того, как значения гиперпараметров влияют на точность - это очень важный для использования нейронных сетей навык, который нужно отрабатывать на практике. Ниже вы должны првоести эксперименты с различными значениями гиперпараметров, включая количество скрытых слоев, скорость обучения, количество обучающих эпох и коэффициент регуляризации. Также вы можете поработать с коэффициентом уменьшения скорости обучения (опционально).

**Ожидаемый результат:** Ваша цель - достичь точности классификации не менее 48%. Луший результат, который мне удалось достичь - 52-53%. Возможно, у вас выйдет еще лучше. Если вам интересно, вы можете попробовать усовершенствовать сеть, например, убрать некоторые изменения в исходных данных, или использовать нормализацию данных (добавляется как слой сети).

In [ ]:
best_net = None # сохраните в этой переменной лучшую модель

#################################################################################
# TODO: Найдите гиперпапаметры, используя проверочную выборку. Сохраните лучшую #
# обученную модель в перменной best_net.                                        #
#                                                                               #
# Для того, чтобы отладить сеть вам, возможно, будет полезно использовать       #
# графики точности и потерь, которые мы видели выше. Графики для данной модели  #
# будут значительно отличаться от графиков "игрушечной" модели, которую мы      #
# использовали ранее.                                                           #
#                                                                               #
# Совет: генерируйте несколько значений для каждого гиперпараметра и напишите   #
# код, который перебирает возможные сочетания для параметров (посмотрите, какие #
# функции Python можно для этого использовать).                                 #
#################################################################################
# *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

pass

# *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****


**Объясните, как вы подбирали гиперпараметры, и какие результаты вы получили (с пояснением, почему именно эти гиперпараметры вы считаете наиболее подходящими)**

$\color{blue}{\textit Ваш ответ:}$

In [ ]:
# Выведите точность на проверочной выборке - она должна быть больше 48%
val_acc = (best_net.predict(X_val) == y_val).mean()
print('Validation accuracy: ', val_acc)

In [ ]:
# Визуализируйте веса лучшей сети
show_net_weights(best_net)

# Запуск сети на тестовой выборке
Когда вы закончили с экспериментами, вам стоит оценть точность классификатора на тестовой выборке. Вы должны получить точность больше 48%.

In [ ]:
# Print your test accuracy: this should be above 48%
test_acc = (best_net.predict(X_test) == y_test).mean()
print('Test accuracy: ', test_acc)